In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import json
import os
import logging
import requests

import functools as ft

from tqdm import tqdm_notebook

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

%load_ext autoreload
%autoreload 2

logging.basicConfig(level=logging.INFO)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
KUCHIKOMI_API = 'https://api.gnavi.co.jp/PhotoSearchAPI/v3/'

# APIの概要を確認

仕様は[こちら](https://api.gnavi.co.jp/api/manual/)

In [21]:
# 機密情報をうっかり流出させないように、別ファイルに保存してあるものを取得
with open('./../gurunabi_api.json','r') as file: 
    api_params=json.load(file)

In [96]:
kuchikomi_params = api_params.copy()
# 取得できるのは最大1000件のみ
# 広すぎると上限数に引掛かかるので300m以内に設定
kuchikomi_params['range'] = 1
kuchikomi_params['latitude'] = '35.681236'
kuchikomi_params['longitude'] = '139.767125'
kuchikomi_params['offset_page'] = '1'

In [97]:
@ft.lru_cache()
def get_kuchikomi_datas():
    data = requests.get(
        KUCHIKOMI_API,
        params = kuchikomi_params
    ).json()
    
    return data

In [98]:
def kuchikomi2df(kuchikomi_data):
    res = kuchikomi_data['response']
    commnts  = []
    num_comment = len(res) -3
    
    for i in range(num_comment):
        com_df = pd.io.json.json_normalize(res[str(i)])
        commnts.append(com_df)
    
    return pd.concat(commnts, ignore_index=True)
    

In [100]:
kuchikomi2df(get_kuchikomi_datas()).head()

,photo.areaname_l,photo.areaname_m,photo.areaname_s,photo.category,photo.comment,photo.distance,photo.image_url.url_1024,photo.image_url.url_200,photo.image_url.url_250,photo.image_url.url_320,photo.latitude,photo.longitude,photo.menu_finish_flag,photo.menu_id,photo.menu_name,photo.messages.first_shop_message.message_body,photo.messages.first_shop_message.send_date,photo.messages.shop_message_count,photo.messages.user_message_count,photo.nickname,photo.photo_genre_id,photo.photo_genre_name,photo.photo_scene_id,photo.photo_scene_name,photo.prefname,photo.shop_id,photo.shop_name,photo.shop_url,photo.total_score,photo.umaso_count,photo.update_date,photo.vote_id
0,東京駅・丸の内・日本橋,東京駅（八重洲）,東京駅（八重洲）,ステーキハンバーグ,すね肉のコリコリした食感が美味。,59,https://mr.gnavi.co.jp/cont/menu_image/88/53/1...,https://mr.gnavi.co.jp/cont/menu_image/88/53/1...,https://mr.gnavi.co.jp/cont/menu_image/88/53/1...,https://mr.gnavi.co.jp/cont/menu_image/88/53/1...,35.681596,139.767608,0,25622,すねバーグ,,,0,0,ぐるなび会員,1,料理・ドリンク,,,PREF13:東京都,1019663,ＳＴＥＡＫ ＆ ＨＡＭＢＵＲＧＶｉｍｏｎ,https://r.gnavi.co.jp/m1vmb2mw0000/?ak=7qF1fl1...,3.0,0,2011-01-25T19:43:46+09:00,169384
1,東京駅・丸の内・日本橋,東京駅（八重洲）,東京駅（八重洲）,ステーキハンバーグ,友人に連れられて伺いました。\r\nボリュームのある粗引き肉のハンバーグで、タルタルステーキ...,59,https://mr.gnavi.co.jp/cont/menu_image/88/53/3...,https://mr.gnavi.co.jp/cont/menu_image/88/53/3...,https://mr.gnavi.co.jp/cont/menu_image/88/53/3...,https://mr.gnavi.co.jp/cont/menu_image/88/53/3...,35.681596,139.767608,0,262317,Ｂランチ とろバーグ,,,0,0,puntopunto,1,料理・ドリンク,,,PREF13:東京都,1019663,ＳＴＥＡＫ ＆ ＨＡＭＢＵＲＧＶｉｍｏｎ,https://r.gnavi.co.jp/m1vmb2mw0000/?ak=7qF1fl1...,3.0,0,2011-10-05T22:14:31+09:00,356692
2,東京駅・丸の内・日本橋,東京駅（八重洲）,東京駅（八重洲）,ステーキハンバーグ,肉汁タップリのジューシーなバーグ。\r\n肉自体に味がシッカリ付いているので、そのまま食べて...,59,https://mr.gnavi.co.jp/cont/menu_image/88/53/3...,https://mr.gnavi.co.jp/cont/menu_image/88/53/3...,https://mr.gnavi.co.jp/cont/menu_image/88/53/3...,https://mr.gnavi.co.jp/cont/menu_image/88/53/3...,35.681596,139.767608,0,266176,Ａランチ すねバーグ,,,0,0,K_S,1,料理・ドリンク,,,PREF13:東京都,1019663,ＳＴＥＡＫ ＆ ＨＡＭＢＵＲＧＶｉｍｏｎ,https://r.gnavi.co.jp/m1vmb2mw0000/?ak=7qF1fl1...,3.0,0,2011-11-05T22:45:27+09:00,379070
3,東京駅・丸の内・日本橋,東京駅（八重洲）,東京駅（八重洲）,ステーキハンバーグ,ハンバーグの挽肉が荒引きで美味しいです。ステーキを頂いてるみたい。,59,https://mr.gnavi.co.jp/cont/menu_image/88/53/3...,https://mr.gnavi.co.jp/cont/menu_image/88/53/3...,https://mr.gnavi.co.jp/cont/menu_image/88/53/3...,https://mr.gnavi.co.jp/cont/menu_image/88/53/3...,35.681596,139.767608,0,262317,Ｂランチ とろバーグ,,,0,0,kyon1120,1,料理・ドリンク,,,PREF13:東京都,1019663,ＳＴＥＡＫ ＆ ＨＡＭＢＵＲＧＶｉｍｏｎ,https://r.gnavi.co.jp/m1vmb2mw0000/?ak=7qF1fl1...,5.0,0,2011-11-15T20:06:14+09:00,398502
4,東京駅・丸の内・日本橋,東京駅（八重洲）,東京駅（八重洲）,ステーキハンバーグ,しっかりとお肉の味がするハンバーグ！付け合わせの野菜もおいしい！,59,https://mr.gnavi.co.jp/cont/menu_image/88/53/4...,https://mr.gnavi.co.jp/cont/menu_image/88/53/4...,https://mr.gnavi.co.jp/cont/menu_image/88/53/4...,https://mr.gnavi.co.jp/cont/menu_image/88/53/4...,35.681596,139.767608,0,266176,Ａランチ すねバーグ,,,0,0,mamikko,1,料理・ドリンク,,,PREF13:東京都,1019663,ＳＴＥＡＫ ＆ ＨＡＭＢＵＲＧＶｉｍｏｎ,https://r.gnavi.co.jp/m1vmb2mw0000/?ak=7qF1fl1...,5.0,0,2011-11-29T23:15:50+09:00,430506


In [144]:
@ft.lru_cache()
def fetch_kuchikomi_by_lat_lot(lat='35.681236',  lot='139.767125'):
    """指定した緯度経度を中心として、口コミを取得"""
    
    use_columns = ['photo.areaname_l', 'photo.areaname_m', 'photo.areaname_s',
       'photo.category', 'photo.comment', 'photo.distance',
       'photo.image_url.url_1024', 'photo.image_url.url_200',
       'photo.image_url.url_250', 'photo.image_url.url_320', 'photo.latitude',
       'photo.longitude', 'photo.menu_finish_flag', 'photo.menu_id',
       'photo.menu_name', 'photo.messages.first_shop_message.message_body',
       'photo.messages.first_shop_message.send_date',
       'photo.messages.shop_message_count',
       'photo.messages.user_message_count', 'photo.nickname',
       'photo.photo_genre_id', 'photo.photo_genre_name',
       'photo.photo_scene_id', 'photo.photo_scene_name', 'photo.prefname',
       'photo.shop_id', 'photo.shop_name', 'photo.shop_url',
       'photo.total_score', 'photo.umaso_count', 'photo.update_date',
       'photo.vote_id']
    
    kuchikomi_params = api_params.copy()
    # 取得できるのは最大1000件のみ
    # 広すぎると上限数に引掛かかるので300m以内に設定
    kuchikomi_params['range'] = 1
    kuchikomi_params['latitude'] = lat
    kuchikomi_params['longitude'] = lot
    kuchikomi_params['hit_per_page'] = 50

    coms_df_list = []
    kuchikomi_params['offset_page'] = 1
    init_json = requests.get(
                KUCHIKOMI_API,
                params = kuchikomi_params
                ).json()
    total_hit_cnt = int(init_json['response']['total_hit_count'])
    total_page_num = total_hit_cnt//50 
    coms_df_list.append(kuchikomi2df(init_json)) 
    print(total_hit_cnt)
    if int(total_hit_cnt) > 1000:
        print('上位千件のみ取得')
             
    for current_num_page in range(2, total_page_num+2):    
        kuchikomi_params['offset_page'] = current_num_page
        coms_df = kuchikomi2df( 
                requests.get(
                KUCHIKOMI_API,
                params = kuchikomi_params
                ).json())
        print(current_num_page)
        
        coms_df_list.append(coms_df)

    return pd.concat(coms_df_list, ignore_index=True)[use_columns]

In [145]:
data = fetch_kuchikomi_by_lat_lot()

/Users/hiroaki/.pyenv/versions/3.7.1/envs/pycoon371/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # Remove the CWD from sys.path while we load stuff.


713
2
3
4
5
6
7
8
9
10
11
12
13
14
15


In [146]:
data.head

,photo.areaname_l,photo.areaname_m,photo.areaname_s,photo.category,photo.comment,photo.distance,photo.image_url.url_1024,photo.image_url.url_200,photo.image_url.url_250,photo.image_url.url_320,photo.latitude,photo.longitude,photo.menu_finish_flag,photo.menu_id,photo.menu_name,photo.messages.first_shop_message.message_body,photo.messages.first_shop_message.send_date,photo.messages.shop_message_count,photo.messages.user_message_count,photo.nickname,photo.photo_genre_id,photo.photo_genre_name,photo.photo_scene_id,photo.photo_scene_name,photo.prefname,photo.shop_id,photo.shop_name,photo.shop_url,photo.total_score,photo.umaso_count,photo.update_date,photo.vote_id
0,東京駅・丸の内・日本橋,東京駅（八重洲）,東京駅（八重洲）,ステーキハンバーグ,すね肉のコリコリした食感が美味。,59,https://mr.gnavi.co.jp/cont/menu_image/88/53/1...,https://mr.gnavi.co.jp/cont/menu_image/88/53/1...,https://mr.gnavi.co.jp/cont/menu_image/88/53/1...,https://mr.gnavi.co.jp/cont/menu_image/88/53/1...,35.681596,139.767608,0.0,25622.0,すねバーグ,,,0,0,ぐるなび会員,1,料理・ドリンク,,,PREF13:東京都,1019663,ＳＴＥＡＫ ＆ ＨＡＭＢＵＲＧＶｉｍｏｎ,https://r.gnavi.co.jp/m1vmb2mw0000/?ak=7qF1fl1...,3.0,0.0,2011-01-25T19:43:46+09:00,169384
1,東京駅・丸の内・日本橋,東京駅（八重洲）,東京駅（八重洲）,ステーキハンバーグ,友人に連れられて伺いました。\r\nボリュームのある粗引き肉のハンバーグで、タルタルステーキ...,59,https://mr.gnavi.co.jp/cont/menu_image/88/53/3...,https://mr.gnavi.co.jp/cont/menu_image/88/53/3...,https://mr.gnavi.co.jp/cont/menu_image/88/53/3...,https://mr.gnavi.co.jp/cont/menu_image/88/53/3...,35.681596,139.767608,0.0,262317.0,Ｂランチ とろバーグ,,,0,0,puntopunto,1,料理・ドリンク,,,PREF13:東京都,1019663,ＳＴＥＡＫ ＆ ＨＡＭＢＵＲＧＶｉｍｏｎ,https://r.gnavi.co.jp/m1vmb2mw0000/?ak=7qF1fl1...,3.0,0.0,2011-10-05T22:14:31+09:00,356692
2,東京駅・丸の内・日本橋,東京駅（八重洲）,東京駅（八重洲）,ステーキハンバーグ,肉汁タップリのジューシーなバーグ。\r\n肉自体に味がシッカリ付いているので、そのまま食べて...,59,https://mr.gnavi.co.jp/cont/menu_image/88/53/3...,https://mr.gnavi.co.jp/cont/menu_image/88/53/3...,https://mr.gnavi.co.jp/cont/menu_image/88/53/3...,https://mr.gnavi.co.jp/cont/menu_image/88/53/3...,35.681596,139.767608,0.0,266176.0,Ａランチ すねバーグ,,,0,0,K_S,1,料理・ドリンク,,,PREF13:東京都,1019663,ＳＴＥＡＫ ＆ ＨＡＭＢＵＲＧＶｉｍｏｎ,https://r.gnavi.co.jp/m1vmb2mw0000/?ak=7qF1fl1...,3.0,0.0,2011-11-05T22:45:27+09:00,379070
3,東京駅・丸の内・日本橋,東京駅（八重洲）,東京駅（八重洲）,ステーキハンバーグ,ハンバーグの挽肉が荒引きで美味しいです。ステーキを頂いてるみたい。,59,https://mr.gnavi.co.jp/cont/menu_image/88/53/3...,https://mr.gnavi.co.jp/cont/menu_image/88/53/3...,https://mr.gnavi.co.jp/cont/menu_image/88/53/3...,https://mr.gnavi.co.jp/cont/menu_image/88/53/3...,35.681596,139.767608,0.0,262317.0,Ｂランチ とろバーグ,,,0,0,kyon1120,1,料理・ドリンク,,,PREF13:東京都,1019663,ＳＴＥＡＫ ＆ ＨＡＭＢＵＲＧＶｉｍｏｎ,https://r.gnavi.co.jp/m1vmb2mw0000/?ak=7qF1fl1...,5.0,0.0,2011-11-15T20:06:14+09:00,398502
4,東京駅・丸の内・日本橋,東京駅（八重洲）,東京駅（八重洲）,ステーキハンバーグ,しっかりとお肉の味がするハンバーグ！付け合わせの野菜もおいしい！,59,https://mr.gnavi.co.jp/cont/menu_image/88/53/4...,https://mr.gnavi.co.jp/cont/menu_image/88/53/4...,https://mr.gnavi.co.jp/cont/menu_image/88/53/4...,https://mr.gnavi.co.jp/cont/menu_image/88/53/4...,35.681596,139.767608,0.0,266176.0,Ａランチ すねバーグ,,,0,0,mamikko,1,料理・ドリンク,,,PREF13:東京都,1019663,ＳＴＥＡＫ ＆ ＨＡＭＢＵＲＧＶｉｍｏｎ,https://r.gnavi.co.jp/m1vmb2mw0000/?ak=7qF1fl1...,5.0,0.0,2011-11-29T23:15:50+09:00,430506
5,東京駅・丸の内・日本橋,東京駅（八重洲）,東京駅（八重洲）,カフェ,,68,https://mr.gnavi.co.jp/cont/menu_image/22/98/3...,https://mr.gnavi.co.jp/cont/menu_image/22/98/3...,https://mr.gnavi.co.jp/cont/menu_image/22/98/3...,https://mr.gnavi.co.jp/cont/menu_image/22/98/3...,35.681788,139.766774,0.0,239420.0,フルーツパフェ,,,0,0,toe,1,料理・ドリンク,,,PREF13:東京都,6125159,INODA COFFEE 東京大丸支店,https://r.gnavi.co.jp/smkeby4m0000/?ak=7qF1fl1...,3.0,0.0,2011-02-07T13:07:33+09:00,323298
6,東京駅・丸の内・日本橋,東京駅（八重洲）,東京駅（八重洲）,サンドイッチ・パン屋,中のりんごがちょうど良いかたさで美味しいです。酸味がほのかにあって甘すぎないところもGOOD！,68,https://mr.gnavi.co.jp/cont/menu_image/86/20/3...,https://mr.gnavi.co.jp/cont/menu_image/86/20/3...,https://mr.gnavi.co.jp/cont/menu_image/86/20/3...,https://mr.gnavi.co.jp/cont/menu_image/86/20/3...,35.681788,139.766774,0.0,281191.0,アップルパイ,,,0,0,ぐるなび会員,1,料理・ドリンク,,,PREF13:東京都,6288037,カーラ アウレリア 大丸東京店,https://r.gnavi.co.jp/mvn8jvxh0000/?ak=7qF1fl1...,3.5,0.0,2011-11-09T09:34:19+09:00,385852
7